In [3]:
(define π 3.14)

π

3.14

In [9]:
(define (square x) (* x x))

(assert = (square 2) 4)
(assert = (square 3) 9)

ok

In [11]:
(define (circle-area r)
  (* π (square r)))

(assert = (circle-area 1) π)
(assert = (circle-area 2) 12.56)

ok